#### Import libraries

---


In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import csv

### Read the Data and create the index

---



##### Define functions necessary

---



In [34]:
import csv

def read_data():
  global df, reverse_index
  df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/search engine/output_for_video.csv", index_col=0)
  # Read the CSV file back into a dictionary
  reverse_index = {}
  with open('/content/drive/MyDrive/Colab Notebooks/search engine/reverse_index.csv', newline='') as csvfile:
      reader = csv.reader(csvfile)
      for row in reader:
          key = row[0]
          values = set(int(x) for x in row[1:])
          reverse_index[key] = values
  data_index_=df["data_index"]
  reverse_index = invert_index(data_index_, reverse_index)
  return df, reverse_index

In [35]:
def invert_index(data_index_, reverse_index=None):
  import csv
  if reverse_index is None:
      reverse_index = {}

  for idx, data in data_index_.items():
      data = data.replace("'", '').replace(" ", '').replace("[", '').replace("]", '')
      data2 = data.split(",")
      for term in data2:
          if term in reverse_index:
              if int(idx) not in reverse_index[term]:
                  reverse_index[term].add(int(idx))
          else:
              reverse_index[term] = {int(idx)}

  # Save the reverse_index dictionary to a CSV file
  with open('/content/drive/MyDrive/Colab Notebooks/search engine/reverse_index.csv', 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      for key, value in reverse_index.items():
          writer.writerow([key, *value])
  return reverse_index

In [ ]:
read_data()

In [8]:
def preprocess(text):
  # Remove non-word characters and convert to lowercase
  text = re.sub(r"\W+", " ", text).lower()
  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text).strip()
  # Tokenize the text
  tokens = word_tokenize(text)
  # Remove stop words
  tokens = [token for token in tokens if token not in stopwords.words("english")]
  # Stem the tokens using Porter stemmer
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  # Join the stemmed tokens back into a string
  preprocessed_text = " ".join(stemmed_tokens)
  preprocessed_text = list(preprocessed_text.split(" "))
  return preprocessed_text

In [40]:
def ranked_retrieval(query):
    query_terms = preprocess(query)
    document_scores = {}
    print(query_terms)
    
    for term in query_terms:
        if term in reverse_index:
            for doc_id in reverse_index[term]:
                if doc_id in document_scores:
                    document_scores[doc_id] += 1
                else:
                    document_scores[doc_id] = 1
    
    sorted_docs = sorted(document_scores.items(), key=lambda x: x[1], reverse=True)
    
    results = []
    for doc_id, score in sorted_docs:
        results.append(df.loc[doc_id])
    
    return results

## Beggining of flask application 

---



In [ ]:
from flask import Flask, request, render_template, url_for
!pip install pyngrok
!pip install flask_ngrok
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
import numpy as np
from pyngrok import ngrok
ngrok.set_auth_token("2NTh3giigpo9d3QwfDZo5naqkLp_6wUENGcBqHQfTKRvs51R2") 


In [63]:
app = Flask(__name__,
            template_folder='/content/drive/MyDrive/Colab Notebooks/search engine/templates',
            static_url_path='', 
            static_folder='/content/drive/MyDrive/Colab Notebooks/search engine/static')
app.config['SECRET_KEY'] = '02de43c128314a2fbdacf46e0f205687fa87ccf09e48b9d8'

run_with_ngrok(app)   
  
@app.route("/", methods=["POST", "GET"])
def home():
    searchQuery = ""
    results_query = ""
    if request.method == "POST":
        results_query = ranked_retrieval(request.form['query_to_search'])
        
    print(searchQuery)
    return render_template('index.html', results_query = results_query)

if __name__ == '__main__':
    read_data()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://8e67-104-198-79-99.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 14:22:00] "GET / HTTP/1.1" 200 -


INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 14:22:00] "GET /clipart.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 14:22:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 14:22:05] "POST / HTTP/1.1" 200 -


['alireza']



INFO:werkzeug:127.0.0.1 - - [31/Mar/2023 14:22:10] "GET /clipart.png HTTP/1.1" 200 -


## For the crawler

---



In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def get_number_pages():
  global number_of_pages
  #scrape the website innitialy to get the number of pages 
  URL = "https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/"
  page = requests.get(URL)
  soup_temp = BeautifulSoup(page.content, "html.parser")
  #get the number of pages
  number_pages = soup_temp.find("nav", class_="pages").find_all("li")
  # -1 because the nav tag class "pages" also includes the "previous page" button
  number_of_pages = len(number_pages)-1
  return number_of_pages

In [8]:
def preprocess(text):
  # Remove non-word characters and convert to lowercase
  text = re.sub(r"\W+", " ", text).lower()
  # Remove extra whitespace
  text = re.sub(r"\s+", " ", text).strip()
  # Tokenize the text
  tokens = word_tokenize(text)
  # Remove stop words
  tokens = [token for token in tokens if token not in stopwords.words("english")]
  # Stem the tokens using Porter stemmer
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  # Join the stemmed tokens back into a string
  preprocessed_text = " ".join(stemmed_tokens)
  preprocessed_text = list(preprocessed_text.split(" "))
  return preprocessed_text

In [9]:
def crawl():
  #create a pandas dataframe and an empty list to store the data before storing as csv
  df=pd.DataFrame(columns=['title', 'authors', 'date', 'keywords', 'abstract', 'title_url', 'url_authors', 'data_index'])
  list_url_authors=[]
  get_number_pages()
  i=0
  #for loop to iterate through the pages of articles
  for i in range(number_of_pages):
    #scrape the page
    url="https://pureportal.coventry.ac.uk/en/organisations/research-centre-for-computational-science-and-mathematical-modell/publications/?page="+str(i)
    page = requests.get(url)
    print(url)
    soup = BeautifulSoup(page.content, "html.parser")
    #get the publications div
    ultag = soup.find("ul", class_="list-results")
    publication_div = ultag.find_all("div", class_="result-container")

    #for every publication: get title, date and title url
    for div in publication_div:
      title = div.find("h3", class_="title" ).find("a", href = True).find("span").text
      title_url = div.find("h3", class_="title" ).find("a", href = True).get("href")
      date = div.find("span", class_="date").text
      print(title)
      print()

      #scrape the page of each publication 
      page_title = requests.get(title_url)
      soup_publications = BeautifulSoup(page_title.content, "html.parser")

      #and get the authors as text
      authors = soup_publications.find("p", class_="relations persons").text
      abstract = soup_publications.find("div", class_="textblock").text if soup_publications.find("div",class_="textblock") else ""
      keywords=[child.find("span").text
                      for tag in [soup_publications.find('ul',{'class': "relations keywords"})]
                      if tag
                      for child in tag.findChildren('li')]
      # for every section containing the authors find the children which are <a> tag
      authors_links=soup_publications.find("p", class_="relations persons")
      children = authors_links.findChildren("a" , recursive=False)
      # for every <a> tag get the link (pureportal profile for every author)
      for child in children:
        link = child.get("href")
        list_url_authors.append(link)
      
      authors_split=authors.split(", ")
      new_list_url_authors=[]  

      for author in authors_split:
        author_name_from_url = author.replace(" ", "-").lower()
        #james-may
        author_found = False
        for url in list_url_authors:
            if author_name_from_url in url.lower():
                new_list_url_authors.append(url)
                author_found = True
                break
        
        if not author_found:
            new_list_url_authors.append(None)            
      #prepare data for index - join title, authors and keywords in one, and remove all non alfabetic charachters, and remove double whitespaces
      data_for_index = title + " " + authors + " " + abstract + " " + " ".join(keywords)
      final_data_for_index = preprocess(data_for_index)
      
      #append the data for each publication to the dataframe: title, list of authors, date, title url and pupeportal profile for the authors
      new_row=pd.Series({'title': title, 'authors': authors, 'date': date, 'keywords': keywords, 'abstract': abstract, 'title_url':title_url, 'url_authors':new_list_url_authors, 'data_index':final_data_for_index})
      df=pd.concat([df, new_row.to_frame().T], ignore_index=True)
      #reset the list after each itteration
      list_url_authors=[]
      new_list_url_authors=[]
    #in the end of a page increase i to scrape the next page of publications  
    i+=1
  store_to_disk(df)


In [10]:
def store_to_disk(df):  
  df.to_csv("/content/drive/MyDrive/Colab Notebooks/search engine/output_for_video.csv")

In [ ]:
crawl()

In [64]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/search engine/output_for_video.csv", index_col=0)

# Create a dictionary with every author and the number of publications they are a part of
authors_dict = {}
for row in df.itertuples():
    authors = row.authors.split(',')
    for author in authors:
        if author.strip() not in authors_dict:
            authors_dict[author.strip()] = 1
        else:
            authors_dict[author.strip()] += 1

# Print the dictionary
print(len(authors_dict))
print(authors_dict)

590
{'Miao Lin Pay': 1, 'Jesper Christensen': 1, 'Fei He': 14, 'Laura Roden': 2, 'Hafiz Ahmed': 1, 'Mathias Foo': 2, 'Majdi Fanous': 3, 'Jonathan Eden': 1, 'Renji Remesan': 2, 'Alireza Daneshkhah': 33, 'Maria Tariq': 2, 'Vasile Palade': 47, 'YingLiang Ma': 7, 'Abdulrahman Altahhan': 1, 'Qi You': 4, 'Chao Li': 7, 'Jun Sun': 10, 'Feng Pan': 2, 'Xiaorui Jiang': 8, 'Junjun Liu': 1, 'Sivasharmini Ganeshamoorthy': 1, 'Dominik Klepl': 6, 'Jonathan M. Eden': 1, 'Shenal Rajintha Alexander Samarathunge Gunawardena': 1, 'Ptolemaios G. Sarrigiannis': 3, 'D. J. Blackburn': 1, 'F. He': 1, 'Huma Shah': 3, 'Fred Roberts': 1, 'Ran Zhao': 1, 'Xinxin Dai': 1, 'Pengpeng Hu': 1, 'Adrian Munteanu': 1, 'Gergely Röst': 1, 'AmirHosein Sadeghimanesh': 10, 'Anas Charroud': 2, 'Karim El El Moutaouakil': 1, 'Ali Yahyaouy': 2, 'Nandor Verba': 5, 'Jonathan Daniel Nixon': 3, 'Elena Gaura': 14, 'Leonardo Alves Dias': 1, 'Alison Halford': 18, 'Anousouya Devi Magaraja': 1, 'Ezhilarasie Rajapackiyam': 1, 'Vaitheki Kanaga

In [65]:
import schedule 
import time
def set_scheduler():
  schedule.every(7).days.do(crawl)

  while True:
      schedule.run_pending()
      time.sleep(1)

In [ ]:
set_scheduler()

In [15]:
!pip install schedule

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
